## 01) Análise literária: 
- Definição dos códigos para as pesquisas literárias, uso dos módulos: Bio e habanero.

In [1]:
pip install Bio

  Obtaining dependency information for Bio from https://files.pythonhosted.org/packages/34/bd/5077f1d9f5a9508edae77a9647228a7304457d22469b2fe88ec7c990f10f/bio-1.6.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
     --- ------------------------------------ 0.2/2.7 MB 6.3 MB/s eta 0:00:01
     ------- -------------------------------- 0.5/2.7 MB 6.2 MB/s eta 0:00:01
     ----------- ---------------------------- 0.8/2.7 MB 6.4 MB/s eta 0:00:01
     --------------- ------------------------ 1.1/2.7 MB 6.1 MB/s eta 0:00:01
     -------------------- ------------------- 1.4/2.7 MB 6.4 MB/s eta 0:00:01
     ------------------------ --------------- 1.7/2.7 MB 6.3 MB/s eta 0:00:01
     --------------------------- ------------ 1.9/2.7 MB 6.3 MB/s eta 0:00:01
     --------------------------------- ------ 2.3/2.7 MB 6.5 MB/s eta 0:00:01
     ------------------------------------- -- 2.6/2.7 MB 6.5 MB/s eta 0:00:01
     -------------------------------

In [18]:
# Função que procura o termo de interesse na database de interesse:
from Bio import Entrez
Entrez.email = 'A.N.Other@example.com'
def buscar_artigos(term, max_artigos):
    """
    Esta função tem como input o termo de busca e como output devolve uma lista de IDs dos artigos encontrados e o nº de artigos encontrados
    """
    links = Entrez.esearch(db="pubmed", retmax = max_artigos, term=term)	
    l = Entrez.read(links)
    ids = l.get("IdList")
    return {"Nº de artigos encontrados": l.get("Count"), "ID dos artigos": ids}


In [4]:
pip install habanero

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Função auxiliar para determinar o URL do artigo:
from habanero import Crossref 
def determinar_url(doi):
  cr = Crossref()
  article = cr.works(ids="10.1101/2023.11.20.567899").get("message")
  return article['URL']

In [21]:
# Função que retorna o título do artigo, data de publicação, nome dos autores e o URL do documento
from Bio import Medline
def dados_artigos(ids_art):
    """
    O input é o ID do artigo de interesse e retorna o título do artigo, data de publicação, nome dos autores e o URL do documento
    """
    handle = Entrez.efetch(db="pubmed", id=ids_art, rettype="medline", retmode="text")
    lista = Medline.parse(handle)

    for r in lista:
        print("------------------")
        print("Título do Artigo:", r.get("TI"))
        print("Data de publicação:", r.get("DP"))
        print("Autores:", r.get("AU"))
        doi = r.get("LID")

        for x,y in enumerate(doi.split()):
           if y == "[doi]":
            print("URL:", determinar_url(doi.split()[x-1]))
        print("------------------")

#### *Resultados da busca literária para os genes de interesse:*
- Genes de busca: IL13, STAT6, IL10 e TGFB1.
- A busca foi realizada para 20 artigos para cada gene de interesse.

<span style='color: red;'>Os resultados alteram a partir da data de pesquisa.</span>

In [16]:
#Termos de pesquisa:
terms = ["IL13", "STAT6", "IL10", "TGFB1"]

In [19]:
# Resultados das pesquisa de no máximo 20 artigos para cada termo:
term1 = buscar_artigos(terms[0], 20)
term2 = buscar_artigos(terms[1], 20)
term3 = buscar_artigos(terms[2], 20)
term4 = buscar_artigos(terms[3], 20)

print("Pesquisa para o gene {} \n".format(terms[0]), term1, "\n","Pesquisa para o gene {} \n".format(terms[1]),term2, "\n","Pesquisa para o gene {} \n".format(terms[2]),term3, "\n","Pesquisa para o gene {} \n".format(terms[3]),term4)

Pesquisa para o gene IL13 
 {'Nº de artigos encontrados': '17241', 'ID dos artigos': ['38097754', '38096593', '38096329', '38094093', '38093354', '38091065', '38088307', '38087324', '38086930', '38085145', '38082117', '38081868', '38081529', '38081297', '38078671', '38078661', '38072390', '38070212', '38061705', '38059621']} 
 Pesquisa para o gene STAT6 
 {'Nº de artigos encontrados': '4303', 'ID dos artigos': ['38098120', '38095318', '38087324', '38082280', '38079450', '38070062', '38069015', '38062486', '38054829', '38044939', '38026128', '38019126', '38016347', '38010398', '37995836', '37986797', '37977069', '37976384', '37974268', '37967817']} 
 Pesquisa para o gene IL10 
 {'Nº de artigos encontrados': '82490', 'ID dos artigos': ['38100544', '38100268', '38099983', '38099943', '38098283', '38098120', '38098024', '38096954', '38096775', '38096529', '38096329', '38095906', '38095633', '38094302', '38094294', '38094290', '38094132', '38094017', '38094010', '38093354']} 
 Pesquisa para

In [ ]:
# Artigos para o gene: IL13
dados_artigos(term1.get("Ids dos artigos definidos"))

In [ ]:
# Artigos para o gene: STAT6
dados_artigos(term2.get("Ids dos artigos definidos"))

In [ ]:
# Artigos para o gene: IL10
dados_artigos(term3.get("Ids dos artigos definidos"))

In [ ]:
# Artigos para o gene: TGFB1
dados_artigos(term4.get("Ids dos artigos definidos"))

## 02) Análise da sequência e das features presentes no NCBI
- Dowload dos genes e criação de documentos com as informações obtidas.
- Os accesion dos genes:
  - TGFB1 : NG_013364
  - IL13  : NG_012090
  - IL10  : NG_012088
  - STAT6 : NG_021272

In [24]:
# Função para o dowload do gene
from Bio import Entrez
from Bio import SeqIO
Entrez.email = 'A.N.Other@example.com'
def dowload_gene(id_gene : str, name_gene : str):
    handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text",id= id_gene)
    for seq_record in SeqIO.parse(handle, "gb"):
        SeqIO.write(seq_record, "{}.gb".format(name_gene), "gb")

In [25]:
# Dowload dos genes de interesse:
dowload_gene("NG_013364", "TGFB1")
dowload_gene("NG_012090", "IL13")
dowload_gene("NG_012088", "IL10")
dowload_gene("NG_021272", "STAT6")